In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from itertools import combinations

In [2]:
def lm_arbitrary_params(split_dataset, coeff):
    X_train, X_test, y_train, y_test = split_dataset
    
    mod = LinearRegression()
    mod.fit(X_train[:,coeff], y_train)
    MSE_train = mean_squared_error(y_train, mod.predict(X_train[:,coeff]))
    MSE_test = mean_squared_error(y_test, mod.predict(X_test[:,coeff]))
    
    return MSE_train, MSE_test, mod.get_params()


def best_selection_at_size(split_dataset, size):
    
    min_MSE_train = 10000
    best_coeff = None
    best_MSE_test = 0
    best_params = None
    
    for comb in combinations(range(20), size):
        train, test, params = lm_arbitrary_params(split_dataset, comb)
        if train < min_MSE_train:
            min_MSE_train = train
            best_MSE_test = test
            best_coeff = comb
            best_params = params
            
    return min_MSE_train, min_MSE_train, best_coeff, best_params

# Conceptual Exercises {-}
## Question 1 {-} 

In [3]:
seed = 20181205
random.seed(seed)

X_blob = make_blobs(n_samples=1000, n_features=20, random_state=seed)[0]

custom_beta = np.array([random.random()*10 for i in range(17)] + [0,0,0])
random.shuffle(custom_beta)


y_response = np.matmul(X_blob, custom_beta) + np.random.normal(0,1,1000)

split_dataset = train_test_split(X_blob, y_response,
                                 test_size=0.9, 
                                 random_state=seed)

In [ ]:
MSE_train_by_size = []
MSE_test_by_size = []
best_coeff_by_size = []
best_params_by_size = []

for i in range(1,21):
    MSE_train, MSE_test, coeff, params = best_selection_at_size(split_dataset, i)
    MSE_train_by_size.append(MSE_train)
    MSE_test_by_size.append(MSE_test)
    best_coeff_by_size.append(coeff)
    best_params_by_size.append(params)

In [ ]:
concept_df = pd.DataFrame({'size': range(1,21),
                          'MSE_train': MSE_train_by_size,
                          'MSE_test': MSE_test_by_size,
                          'coeff': best_coeff_by_size,
                          'params': best_params_by_size})

# dataframe entry with the lowest MSE_train value
concept_df.sort_values(by='MSE_train').iloc[0]

In [ ]:
concept_df.plot(x='size', y='MSE_train', kind='line',
               figsize=(8,6), xticks=range(1,21))
plt.title("Best Model's Train MSE at Each Subset Size", fontsize=20)
plt.xlabel('Predictor Subset Size', fontsize=15)
plt.ylabel('Train MSE', fontsize=15)
plt.legend(fontsize=12);

The model of subset size 16 has the minimum training MSE value.

In [ ]:
# dataframe entry with the lowest MSE_test value
concept_df.sort_values(by='MSE_test').iloc[0]

In [ ]:
concept_df.plot(x='size', y='MSE_test', kind='line',
               figsize=(8,6), xticks=range(1,21))
plt.title("Best Model's Test MSE at Each Subset Size", fontsize=20)
plt.xlabel('Predictor Subset Size', fontsize=15)
plt.ylabel('Test MSE', fontsize=15)
plt.legend(fontsize=12);

The model of subset size 16 has the minimum testing MSE value. We find the the subset size of 16 possesses both the minimal training MSE and the minimal testing MSE.

In [ ]:
# positions of zero entries in custom_beta
[i for i in range(20) if custom_beta[i] == 0]

In [ ]:
# positions of parameters in the best model (subset size = 16)
concept_df['coeff'][15]

In [ ]:
# the parameter left out from the first two sets
custom_beta[5]

The best model (coefficient size = 16) from the best subset selection closely resemble the model we used to generate the X data. This model excludes all zero entries we put into our generating $\beta$ vector, and exclude another non-zero coefficient at position 5 that is very close to zero (0.094) in value.